# 교차검증

In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [49]:
oz = pd.read_csv("ozone.csv")
oz

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5
...,...,...,...,...,...,...
148,30.0,193.0,6.9,70,9,26
149,NaN,145.0,13.2,77,9,27
150,14.0,191.0,14.3,75,9,28
151,18.0,131.0,8.0,76,9,29


In [4]:
oz = oz.dropna()
oz

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
6,23.0,299.0,8.6,65,5,7
...,...,...,...,...,...,...
147,14.0,20.0,16.6,63,9,25
148,30.0,193.0,6.9,70,9,26
150,14.0,191.0,14.3,75,9,28
151,18.0,131.0,8.0,76,9,29


In [26]:
X = oz[["Solar.R", "Wind", "Temp"]]
Y = oz[["Ozone"]]
X = X.astype(int)
Y = Y.astype(int)

In [27]:
X = np.array(X)
Y = np.array(Y)

In [54]:
dt_clf = DecisionTreeClassifier(random_state=156)
dt_r = DecisionTreeRegressor(random_state=156)

In [52]:
# 5개의 폴드세트로 분리하는 KFold 객체와 폴드별 정확도 담을 리스트생성
kfold = KFold(n_splits=5)
cv_accuracy = []
print(X.shape[0])

111


In [43]:
ols_clf = sm.OLS(Y,X)
# result = ols_clf.fit()

In [57]:
n_iter = 0

# 학습용, 검증용 테스트데이터 추출
for train, test in kfold.split(X):
    Xtrain, Xtest = X[train], X[test]
    Ytrain, Ytest = Y[train], Y[test]
    
    # 학습 및 예측
    dt_r.fit(Xtrain,Ytrain)
    pre = dt_r.predict(Xtest)
    n_iter +=1
    
    # 반복마다 정확도 측정
    accuracy = np.round(accuracy_score(Ytest,pre), 4)
    train_size = Xtrain.shape[0]
    test_size = Xtest.shape[0]
    print('\n#{0} 교차 검증 정확도:{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter, test))
    cv_accuracy.append(accuracy)

# iteration병 정확도 합해 평균정확도 계산
print(np.mean(cv_accuracy))


#1 교차 검증 정확도:0.0435, 학습 데이터 크기: 88, 검증 데이터 크기: 23
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]

#2 교차 검증 정확도:0.0455, 학습 데이터 크기: 89, 검증 데이터 크기: 22
#2 검증 세트 인덱스:[23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]

#3 교차 검증 정확도:0.0, 학습 데이터 크기: 89, 검증 데이터 크기: 22
#3 검증 세트 인덱스:[45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66]

#4 교차 검증 정확도:0.0455, 학습 데이터 크기: 89, 검증 데이터 크기: 22
#4 검증 세트 인덱스:[67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88]

#5 교차 검증 정확도:0.0455, 학습 데이터 크기: 89, 검증 데이터 크기: 22
#5 검증 세트 인덱스:[ 89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110]
0.036


In [62]:
# 정규화
from sklearn.preprocessing import MinMaxScaler

# 교차검증용 데이터 나누기
Xtrain, Xtest, ytrain, ytest= train_test_split(X, Y, test_size=0.2, random_state=42)
Val_Xtrain, Val_Xtest, Val_ytrain, Val_ytest = train_test_split(Xtrain,ytrain,test_size=0.2,random_state=42)

MinMax = MinMaxScaler()
MinMax.fit(Val_Xtrain)
Min_train_scaled = MinMax.transform(Val_Xtrain)
Min_test_scaled = MinMax.transform(Val_Xtest)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(Min_train_scaled, Val_ytrain)

LinearRegression()

In [63]:
lr.score(Min_test_scaled, Val_ytest)

0.42882360085672344